In [1]:
import psycopg2 as pg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [2]:
columns_business = [
'business_id',
# 'name',
# 'address',
#'city',
'state',
#'postal_code',
'latitude',
'longitude',
# 'stars',
'review_count',
'attributes_goodforkids',
#'categories',   further processing
'is_open',
# 'hours_monday',
# 'hours_tuesday',
# 'hours_wednesday',
# 'hours_thursday',
# 'hours_friday',
# 'hours_saturday',
# 'hours_sunday',
'attributes_restaurantsreservations',
# 'attributes_goodformeal',
# 'attributes_businessparking',
'attributes_caters',
'attributes_noiselevel',
'attributes_restaurantstableservice',
'attributes_restaurantstakeout',
'attributes_restaurantspricerange2',
'attributes_outdoorseating',
'attributes_bikeparking',
# 'attributes_ambience',
'attributes_hastv',
'attributes_wifi',
'attributes_alcohol',
'attributes_restaurantsattire',
'attributes_restaurantsgoodforgroups',
'attributes_restaurantsdelivery',
'attributes_businessacceptscreditcards',
'attributes_businessacceptsbitcoin',
#'attributes_byappointmentonly',
#'attributes_acceptsinsurance',
# 'attributes_music',
'attributes_goodfordancing',
'attributes_coatcheck',
'attributes_happyhour',
# 'attributes_bestnights',
'attributes_wheelchairaccessible',
'attributes_dogsallowed',
# 'attributes_byobcorkage',
'attributes_drivethru',
'attributes_smoking',
#'attributes_agesallowed',
#'attributes_hairspecializesin',
#'attributes_corkage',
#'attributes_byob',
#'attributes_dietaryrestrictions', further processing
#'attributes_open24hours',
#'attributes_restaurantscounterservice',
'restaurant']

In [3]:
list_as_string =', '.join(columns_business)

In [4]:
con = pg.connect(database="postgres", user="flatiron_user_1", password="K*%4t3VK0ab%gn", host="34.74.239.44", port="5432")
cur = con.cursor()

u_cutoff=87
l_cutoff=10

In [5]:

cur.execute(f"SELECT {list_as_string} FROM business WHERE restaurant IS true AND REVIEW_COUNT < {u_cutoff} AND REVIEW_COUNT > {l_cutoff}")
dataframe=cur.fetchall()

In [6]:
dataframe=pd.DataFrame(dataframe)
dataframe.columns=columns_business


In [7]:
# plt.figure(figsize=(16,10))
# sns.heatmap(dataframe.isna())

In [8]:
np.mean(dataframe.isnull(), axis=0)

business_id                              0.000000
city                                     0.000000
state                                    0.000000
postal_code                              0.000000
latitude                                 0.000000
longitude                                0.000000
review_count                             0.000000
attributes_goodforkids                   0.259743
is_open                                  0.000000
attributes_restaurantsreservations       0.268462
attributes_caters                        0.414482
attributes_noiselevel                    0.350843
attributes_restaurantstableservice       0.761391
attributes_restaurantstakeout            0.153357
attributes_restaurantspricerange2        0.050906
attributes_outdoorseating                0.218061
attributes_bikeparking                   0.213781
attributes_hastv                         0.290473
attributes_wifi                          0.318784
attributes_alcohol                       0.298474


In [9]:
dataframe.nunique()

business_id                              37618
city                                       552
state                                       11
postal_code                               7175
latitude                                 34993
longitude                                34729
review_count                                76
attributes_goodforkids                       3
is_open                                      2
attributes_restaurantsreservations           3
attributes_caters                            3
attributes_noiselevel                        9
attributes_restaurantstableservice           3
attributes_restaurantstakeout                3
attributes_restaurantspricerange2            5
attributes_outdoorseating                    3
attributes_bikeparking                       3
attributes_hastv                             3
attributes_wifi                              7
attributes_alcohol                           7
attributes_restaurantsattire                 7
attributes_re

In [10]:
dataframe.attributes_restaurantsreservations.value_counts()
dataframe.attributes_smoking.value_counts()

u'no'         762
u'outdoor'    636
u'yes'        173
'no'            8
'outdoor'       4
None            4
'yes'           1
Name: attributes_smoking, dtype: int64

In [11]:
list_to_strip=[
    'attributes_alcohol',
    'attributes_restaurantsattire',
    'attributes_wifi',
    'attributes_smoking',
    'attributes_noiselevel',
              ]
dataframe2=dataframe.copy()
for col in list_to_strip:
    dataframe2[col]=dataframe2[col].str.strip("u\'")

In [12]:
for col in columns_business:
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='None')
    dataframe2[col]=dataframe2[col].where(dataframe2[col]!='none')


/home/will/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [13]:
dataframe2.attributes_restaurantstakeout.nunique(dropna=False)

4

In [14]:
dataframe2.attributes_restaurantstakeout.value_counts(dropna=False)


True     29382
NaN       5786
False     2450
Name: attributes_restaurantstakeout, dtype: int64

In [15]:
dataframe2.head()

,business_id,city,state,postal_code,latitude,longitude,review_count,attributes_goodforkids,is_open,attributes_restaurantsreservations,...,attributes_businessacceptscreditcards,attributes_businessacceptsbitcoin,attributes_goodfordancing,attributes_coatcheck,attributes_happyhour,attributes_wheelchairaccessible,attributes_dogsallowed,attributes_drivethru,attributes_smoking,restaurant
0,03bL5yYHb-chhJwyeSAkfQ,Montréal,QC,H3H 2T5,45.4958037,-73.5804942,25,True,1,True,...,None,None,None,None,None,False,False,None,None,True
1,0cWaLXoq8CwSwbThhKBhtQ,Calgary,AB,T3B 0N1,51.07224045,-114.1630555977,11,True,1,True,...,None,None,None,None,None,None,None,None,None,True
2,1PnHQEOXnxosKYhB86Ou7A,Scarborough,ON,M1S 3P6,43.7837428,-79.2531863,24,True,1,False,...,None,None,None,None,None,None,None,None,None,True
3,2JYszoQKgJj6pPbaN7cLwQ,Pittsburgh,PA,15212,40.4458370011,-80.0113584101,34,False,0,True,...,True,False,None,None,None,True,False,None,None,True
4,8jl0zzTGajfSDpdVDPYn6Q,Markham,ON,L6C 1Y9,43.8704482756,-79.361412512,70,True,1,True,...,None,None,None,None,None,None,None,None,None,True


In [16]:
cols=dataframe.columns
cols=cols.drop(['review_count','restaurant','latitude','longitude','business_id'])

In [17]:

dataframe2.latitude=dataframe2.latitude.astype(float)
dataframe2.longitude=dataframe2.longitude.astype(float)

In [18]:
categorical_features=cols
categorical_features

Index(['state', 'attributes_goodforkids', 'is_open',
       'attributes_restaurantsreservations', 'attributes_caters',
       'attributes_noiselevel', 'attributes_restaurantstableservice',
       'attributes_restaurantstakeout', 'attributes_restaurantspricerange2',
       'attributes_outdoorseating', 'attributes_bikeparking',
       'attributes_hastv', 'attributes_wifi', 'attributes_alcohol',
       'attributes_restaurantsattire', 'attributes_restaurantsgoodforgroups',
       'attributes_restaurantsdelivery',
       'attributes_businessacceptscreditcards',
       'attributes_businessacceptsbitcoin', 'attributes_goodfordancing',
       'attributes_coatcheck', 'attributes_happyhour',
       'attributes_wheelchairaccessible', 'attributes_dogsallowed',
       'attributes_drivethru', 'attributes_smoking'],
      dtype='object')

In [19]:
numerical_features=['review_count','latitude','longitude']

In [20]:
preprocess = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(drop='first'), categorical_features))

In [21]:
dataframe2[categorical_features]=dataframe2[categorical_features].fillna(9)


In [22]:
# preprocess.fit_transform(dataframe).toarray()[:5]
dataframe2[categorical_features]=dataframe2[categorical_features].astype(str)

In [23]:
ohe=OneHotEncoder(drop='first')
ohe.fit(dataframe2[categorical_features])

OneHotEncoder(categorical_features=None, categories=None, drop='first',
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [24]:
ohe.transform(dataframe2[categorical_features])

<37618x66 sparse matrix of type '<class 'numpy.float64'>'
	with 479656 stored elements in Compressed Sparse Row format>

In [25]:
dataframe2[categorical_features].head()

,state,attributes_goodforkids,is_open,attributes_restaurantsreservations,attributes_caters,attributes_noiselevel,attributes_restaurantstableservice,attributes_restaurantstakeout,attributes_restaurantspricerange2,attributes_outdoorseating,...,attributes_restaurantsdelivery,attributes_businessacceptscreditcards,attributes_businessacceptsbitcoin,attributes_goodfordancing,attributes_coatcheck,attributes_happyhour,attributes_wheelchairaccessible,attributes_dogsallowed,attributes_drivethru,attributes_smoking
0,QC,True,1,True,True,average,False,True,1,True,...,True,9,9,9,9,9,False,False,9,9
1,AB,True,1,True,9,9,9,True,2,False,...,True,9,9,9,9,9,9,9,9,9
2,ON,True,1,False,9,9,9,True,2,True,...,False,9,9,9,9,9,9,9,9,9
3,PA,False,0,True,True,average,True,True,2,True,...,False,True,False,9,9,9,True,False,9,9
4,ON,True,1,True,False,quiet,9,True,2,False,...,False,9,9,9,9,9,9,9,9,9


In [26]:
preprocess = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OneHotEncoder(drop='first'), categorical_features))
preprocess.fit_transform(dataframe2)

<37618x69 sparse matrix of type '<class 'numpy.float64'>'
	with 592510 stored elements in Compressed Sparse Row format>

In [27]:
pipe = Pipeline([('ct', preprocess),
                 ('clf', LogisticRegression(random_state=42))])

grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
                   'clf__C': [.001,.01,.1,1,10,100,1000],
                  }] 


In [28]:
gridsearch = GridSearchCV(estimator=pipe,
			param_grid=grid_params_lr,
			scoring='accuracy',
            return_train_score=True,
			cv=5)

In [31]:
dataframe2.columns
data=dataframe2.drop(['business_id','restaurant',''])
target

Index(['business_id', 'city', 'state', 'postal_code', 'latitude', 'longitude',
       'review_count', 'attributes_goodforkids', 'is_open',
       'attributes_restaurantsreservations', 'attributes_caters',
       'attributes_noiselevel', 'attributes_restaurantstableservice',
       'attributes_restaurantstakeout', 'attributes_restaurantspricerange2',
       'attributes_outdoorseating', 'attributes_bikeparking',
       'attributes_hastv', 'attributes_wifi', 'attributes_alcohol',
       'attributes_restaurantsattire', 'attributes_restaurantsgoodforgroups',
       'attributes_restaurantsdelivery',
       'attributes_businessacceptscreditcards',
       'attributes_businessacceptsbitcoin', 'attributes_goodfordancing',
       'attributes_coatcheck', 'attributes_happyhour',
       'attributes_wheelchairaccessible', 'attributes_dogsallowed',
       'attributes_drivethru', 'attributes_smoking', 'restaurant'],
      dtype='object')

In [30]:
#X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

NameError: name 'iris' is not defined

In [ ]:
# gridsearch.fit(X_train, y_train);

In [ ]:
cross_val = pd.DataFrame(search.cv_results_)
cross_val2 = cross_val.sort_values('rank_test_score')
cross_val2

In [ ]:
dataframe2